Need to try and get a better MAE as the crurent one is not sufficient enough to warrant a backtest  

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.impute import SimpleImputer

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer

file_path = './portfolio_value_fair_value_data.csv'
df = pd.read_csv(file_path)
df = df.dropna()
df = df.drop(columns=['Ticker'])
df = pd.get_dummies(df)
df = df.apply(pd.to_numeric, errors='coerce')
df = df.dropna()
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()
X = df.drop(columns=['Fair Value'])
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

y = df['Fair Value']


In [10]:
model = LinearRegression()
rfe = RFE(estimator=model, n_features_to_select=25)
# rfe.fit(X_scaled, y)
# selected_features = rfe.support_
# X_selected = X_scaled[:, selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# selected_columns = X.columns[selected_features]
# print("Selected features:", selected_columns)

Mean Squared Error: 252892346.84105197
R-squared: -21253.09065433016


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Load and preprocess the dataset
df = pd.read_csv('./data/portfolio_value_fair_value_data.csv')
df = df.dropna()
df = df.drop(columns=['Ticker'])
df = pd.get_dummies(df)
df = df.apply(pd.to_numeric, errors='coerce')
df = df.dropna()
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df = df.clip(lower=1e-12, upper=1e12)
X = df.drop(columns=['Fair Value'])
y = df['Fair Value']

# Log transformation of skewed features
skewed_feats = X.apply(lambda x: np.abs(x.skew())).sort_values(ascending=False)
skewed_features = skewed_feats[skewed_feats > 0.75].index
for feat in skewed_features:
    if X[feat].dtype in [np.float64, np.float32]:
        X[feat] = np.log1p(X[feat].clip(lower=1e-12))

X = X.fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Feature selection
selection_model = XGBRegressor(objective='reg:squarederror', random_state=42, n_estimators=100)
selection_model.fit(X_train, y_train)
sel = SelectFromModel(selection_model, threshold="mean", prefit=True)
X_train_selected = sel.transform(X_train)
X_test_selected = sel.transform(X_test)
joblib.dump(sel, 'feature_selector.pkl')
selected_features = X.columns[sel.get_support()].tolist()
pd.DataFrame({'feature_name': selected_features}).to_csv('selected_features.csv', index=False)

# Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=kf, scoring='neg_mean_absolute_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

# Save the best model and scaler
best_xgb = grid_search.best_estimator_
joblib.dump(best_xgb, 'best_xgb_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Print best MAE score
best_mae_score = -grid_search.best_score_  # GridSearchCV returns the negative of MAE, so negate it
print(f"Best MAE Score: {best_mae_score:.2f}")


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.7; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=1, n_estimators=100, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=100, subsample=0.9; tot

Best Hyperparameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8} -> MAE of 9.65 dollars


In [12]:
import joblib

# Save the model
joblib.dump(best_xgb, 'best_xgb_model.pkl')

# Save the scaler (StandardScaler)
joblib.dump(scaler, 'scaler.pkl')

# If you have any imputers or other preprocessing objects, save them as well
# Example: imputer = SimpleImputer() if you used one
# joblib.dump(imputer, 'imputer.pkl')


['scaler.pkl']

In [20]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_absolute_error

# Load the necessary files
best_xgb = joblib.load('best_xgb_model.pkl')
scaler = joblib.load('scaler.pkl')
sel = joblib.load('feature_selector.pkl')

# Ensure X_test and y_test are loaded and preprocessed consistently
# Example: X_test should be preprocessed and scaled before feature selection

# Apply scaling to X_test
X_test_scaled = scaler.transform(X_test)

# Apply feature selection to scaled X_test
X_test_selected = sel.transform(X_test_scaled)

# Make predictions on the preprocessed test set
y_pred = best_xgb.predict(X_test_selected)

# Calculate errors
errors = y_test - y_pred
abs_errors = np.abs(errors)

# Create a DataFrame to hold the results
results_df = pd.DataFrame({
    'Actual': y_test.values,  # Ensure y_test is a Pandas Series or convert to values
    'Predicted': y_pred,
    'Error': errors,
    'Absolute Error': abs_errors
})

# Save the results to a CSV file
results_df.to_csv('test_set_errors.csv', index=False)

# Calculate and print the overall MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on the test set: {mae:.2f} dollars")

# Show the first few rows of the results for a quick look
print(results_df.head())


Mean Absolute Error on the test set: 71.53 dollars
       Actual  Predicted       Error  Absolute Error
1271  112.774  -0.116838  112.890838      112.890838
688   121.000   2.462278  118.537722      118.537722
2143   30.415  -1.067092   31.482092       31.482092
2255  147.209   0.949413  146.259587      146.259587
114   139.000   3.492757  135.507243      135.507243


/home/odhran_user/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_absolute_error

# Load the necessary files
best_xgb = joblib.load('best_xgb_model.pkl')
scaler = joblib.load('scaler.pkl')
sel = joblib.load('feature_selector.pkl')

# Ensure X_test and y_test are loaded and preprocessed consistently
# Example: X_test should be preprocessed and scaled before feature selection

# Apply scaling to X_test
X_test_scaled = scaler.transform(X_test)

# Apply feature selection to scaled X_test
X_test_selected = sel.transform(X_test_scaled)

# Make predictions on the preprocessed test set
y_pred = best_xgb.predict(X_test_selected)

# Calculate errors
errors = y_test - y_pred
abs_errors = np.abs(errors)

# Create a DataFrame to hold the results
results_df = pd.DataFrame({
    'Actual': y_test.values,  # Ensure y_test is a Pandas Series or convert to values
    'Predicted': y_pred,
    'Error': errors,
    'Absolute Error': abs_errors
})

# Save the results to a CSV file
results_df.to_csv('test_set_errors.csv', index=False)

# Calculate and print the overall MAE
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error on the test set: {mae:.2f} dollars")

# Show the first few rows of the results for a quick look
print(results_df.head())


Mean Absolute Error on the test set: 71.53 dollars
       Actual  Predicted       Error  Absolute Error
1271  112.774  -0.116838  112.890838      112.890838
688   121.000   2.462278  118.537722      118.537722
2143   30.415  -1.067092   31.482092       31.482092
2255  147.209   0.949413  146.259587      146.259587
114   139.000   3.492757  135.507243      135.507243


/home/odhran_user/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
